In [286]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np

In [222]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [223]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [224]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [225]:
iris_y.rename(columns = {0:4}, inplace = True) 

In [226]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [227]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [228]:
iris.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3],4:"target"}, inplace = True)

In [229]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [230]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [231]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [232]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [233]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [234]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [235]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [236]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [237]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [238]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [239]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [240]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [241]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [242]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
    data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [243]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [244]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [245]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [246]:
best_attribute_c45(iris,'target',False)

('petal width (cm)', 0.8)

In [247]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [248]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [249]:
c45(iris, "target",True)

In [250]:
print_tree(c45(iris, "target",True),0)

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [251]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [252]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [253]:
p=iris

In [254]:
p=p.drop('target',axis=1)

In [255]:
p=p.reset_index().drop('index',axis=1)

In [256]:
p['sepal width (cm)'] = None

In [257]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [258]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [259]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [260]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

In [261]:





############################################          ANN              ###################################################





def sigmoid(x):
    return 1/(1+math.exp(-x))

In [262]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [263]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [264]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [265]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [266]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [267]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [268]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [269]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [270]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [271]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [272]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [273]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [274]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [275]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [276]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [277]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [278]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [309]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


In [280]:




#############################################       Testing                ############################################





In [325]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [342]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)
            


In [281]:
################################# Testing DTL

data10, data90 = get_data_validate(iris)

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')


In [350]:
model = c45(data90, "target",True)
prediction_dtl = pred_c45(data10, model, "target")
print(prediction_dtl)

   target  prediction
0       0           0
1       0           0
2       0           0
3       0           0
4       0           0
5       1           1
6       1           1
7       1           1
8       1           1
9       1           1
10      2           2
11      2           2
12      2           2
13      2           2
14      2           2


In [351]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          5  0  0
1          0  5  0
2          0  0  5


In [352]:
testing_accuracy(prediction_dtl,'target','prediction')

1.0

In [294]:
################################# Testing ANN

data10, data90 = get_data_validate(iris)

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')


In [320]:
hidden_node = [3,4]
x = MLP(data90, 'target', hidden_node,20,0.1)

43.48833568538582
42.70822347567036
42.17112666386392
41.322791769832385
40.18618523469346
38.78060296040001
37.15120618586272
35.38264788976741
33.59117993317013
31.895192919849784
30.380920190747304
29.087278737784914
28.014582946769583
27.14061815364156
26.433710578208675
25.86161998720369
25.396111291572364
25.01424179158375
24.698021115329844
24.4335486568903


[[-0.6520824043288473, -0.12612749884421393, 0.2210515977947524], [0.6633886940261726, -0.3650212436314327, -0.24443709848412093, -0.7575671954059494], [2.839182504468756, -1.6060814401359975, -2.7322495616122895], []]
[[-1.001130841374595, 0.5685113400327318, 1.4587787322843395], [-1.241129851270293, -0.6688519979542902, 0.3280170819873375, 0.7236020441293947], [-1.3362187429670123, -0.7776377873497242, 0.36164543904694624], []]
[[-0.08209304572818477, 0.8290279389938202, -1.669375059576119], [3.3091490573107816, -1.125337720735463, -1.3809977795358372, -2.636232444568098], [-1.809452710828952, -0.46048108644855484, -0.351

In [353]:
prediction_ann = predMyMLP(data10, 'target', x, 3)
print(prediction_ann)

   target  prediction
0       0           0
1       0           0
2       0           0
3       0           0
4       0           0
5       1           2
6       1           2
7       1           2
8       1           2
9       1           2
10      2           2
11      2           2
12      2           2
13      2           2
14      2           2


In [354]:
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          5  0  0
1          0  0  5
2          0  0  5


In [355]:
testing_accuracy(prediction_ann,'target','prediction')

0.67